In [ ]:
import re
from newspaper import Article, ArticleException

def crawl_newspaper(url):
    """기사 링크를 입력으로 받아 제목, 내용 그리고 링크를 파싱하여 dictionary 형태로 반환하는 함수"""
    
    try:
        article = Article(url,
                          language='ko',
                          memorize_articles=False,
                          fetch_images=False,
                          skip_bad_cleaner=True)
        article.download()
        article.parse()

        article_data = {}
        article_data['title'] = article.title.strip()  # 기사 제목
        article_data['text'] = article.text.replace("\n", " ").strip()  # 기사 내용에서 줄바꿈을 지우고, 좌우 공백 제거
        article_data['url'] = url  # 기사 링크

        return article_data
    except ArticleException as AE:
        raise AE
    
    
tmp_article = crawl_newspaper("https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=022&aid=0003376441")

for k, v in tmp_article.items():
    print(k, ":", v[:100])

In [ ]:
a=1
b=1
a

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_newspaper_list(query,  # 검색어
                       date=None,  # 기사 일자
                       start_page=1  # 뉴스 리스트 페이지
                      ):
    """네이버에서 검색어를 이용해 해당 기간 내의 뉴스 리스트를 반환하는 함수"""
    
    url = 'https://search.naver.com/search.naver?&where=news&query={query}&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={start_date}&de={end_date}&start={start_page}'  # query와 기간을 조건으로 한 검색 url 
    if not date:
        date = datetime.now()  # 오늘 날짜 instance 생성
        date = date.strftime("%Y.%m.%d")  # 날자 instance를 문자열 형태로 변환 ex) 2019.06.30
    
    res = requests.get(url.format(query=query, 
                                  start_date=date, 
                                  end_date=date, 
                                  start_page=(start_page - 1) * 10 + 1))
    
    # BeautifulSoup을 이용해 웹 html을 파싱
    soup = BeautifulSoup(res.text, 'lxml')
    
    # 최대 기사 페이지 수를 확인
    pages = [tag.get_text() for tag in soup.select(".paging > a, .paging > strong")] # 이전페이지,1,2,3페이지
    max_page = 0
    for page in pages:
        try:
            page = int(page)
            max_page = max(page, max_page)
        except:
            continue
            
    if start_page > max_page:  # 더이상 페이지가 없을 경우 빈 리스트를 반환
        return []
    
    # html 구조에서 해당 기사로 접근할 수 있는 url을 추출
    link_list = [a['href'] for a in soup.select(".news > ul.type01 > li > dl > dt > a")]  
    return link_list

In [ ]:
get_newspaper_list("딥러닝", "2019.07.01", start_page=1)

In [ ]:
from datetime import datetime, timedelta
from time import time

def crawl_newspaper_by_query(query,  # 검색어 키워드
                             start_date=None,
                             end_date=None,
                             max_page=10):
    
    
    article_list = []  # 기사 리스트
    crawled_url = set()  # 검색한 기사 중복 검색을 위한 변수
        
    if not start_date:
        start_date = datetime.now()  # 오늘 날짜 instance 생성
    else:
        start_date = datetime.strptime(start_date, "%Y.%m.%d")  # 2019.06.30과 같은 문자열을 python instance로 변경
        
    if not end_date:
        end_date = datetime.now()
    else:
        end_date = datetime.strptime(end_date, "%Y.%m.%d")
    
    while start_date < end_date:
        print("Date: ", end_date)
        for cur_page in range(1, max_page):
            print("Page: {}".format(cur_page))
            
            # end_date 날짜의 기사 링크 수집
            link_list = get_newspaper_list(query, end_date.strftime("%Y.%m.%d"), start_page=cur_page)

            if not link_list:  # 더이상 수집할 뉴스가 없을 경우
                break
            
            for link in link_list:
                # 중복 수집 방지
                if link in crawled_url:
                    continue
                else:
                    crawled_url.update([link])
                
                try:
                    article = crawl_newspaper(link)
                    article['date'] = start_date
                    article_list.append(article)
                except ArticleException as AE:
                    continue
        
        end_date = end_date - timedelta(days=1)  # 수집할 날짜를 하루 전으로 변경
    return article_list

In [ ]:
start = time()
articles = crawl_newspaper_by_query("딥러닝", "2019.06.01", "2019.07.01", max_page=10)
print(time() - start)
print("Total articles: {:,}".format(len(articles)))
print("="*20, "Article sample", "="*20)
print(articles[0])

## Text preprocessing

In [282]:
from konlpy.tag import Okt

# https://konlpy-ko.readthedocs.io/ko/v0.4.3/morph/
tagger = Okt()  # Twitter 형태소 분석기 (Open Korean Text)

sentence = "SK텔레콤-LG전자, 5G로 클라우드와 로봇 결합했다… 로봇산업 혁신 기대"
words = tagger.pos(sentence, 
                   norm=True,  # 정규화 - 입니다 ㄹㄹ -> 입니다로 모아주는
                   stem=True)  # 어근화 - 원형으로 바꾸는 작업
for w, t in words:
    print(w, t)
    
"""
정규화 (normalization): 입니닼ㅋㅋ => 입니다 ㅋㅋ
어근화 (stemming): 한국어를 처리하는 예시입니다 => 한국어, 를, 처리, 하다, 예시, 이다
"""

SK Alpha
텔레콤 Noun
- Punctuation
LG Alpha
전자 Noun
, Punctuation
5 Number
G Alpha
로 Noun
클라우드 Noun
와 Josa
로봇 Noun
결합 Noun
하다 Verb
… Punctuation
로봇산업 Noun
혁신 Noun
기대 Noun


'\n정규화 (normalization): 입니닼ㅋㅋ => 입니다 ㅋㅋ\n어근화 (stemming): 한국어를 처리하는 예시입니다 => 한국어, 를, 처리, 하다, 예시, 이다\n'

In [9]:
from collections import Counter

words_list = []
vocab = Counter() # 글자를 숫자로 만들기 위함
tag_set = set(['Noun', 'Alpha'])  # 키워드로 포함하고자 하는 품사 (명사, 알파벳)
stopwords = set(["글자"])  # 키워드로 포함하지 않을 금지 단어 -> 하면서 포함되도록..

for i, article in enumerate(articles):
    if i % 100 == 0:
        print(i)
        
    words = tagger.pos(article['text'], norm=True, stem=True)
    tmp_words = []

    for w, t in words:
        if t in tag_set and len(w) > 1 and w not in stopwords:
            tmp_words.append(w)
    vocab.update(tmp_words)
    words_list.append((tmp_words, article))

NameError: name 'articles' is not defined

In [10]:
print(words_list[0])

IndexError: list index out of range

In [285]:
vocab = sorted([w for w, freq in vocab.most_common(10000)]) # voca size
word2id = {w: i for i, w in enumerate(vocab)}  # {"SK": 0, "로봇": 1, "딥러닝": 2, ...}

new_words_list = []
for words, article in words_list:
    words = [w for w in words if w in word2id]  # vocab에 있는 단어만 사용
    
    # 기사 내 서로 다른 단어 개수가 10개 이상인 기사만 사용
    if len(words) > 10:
        new_words_list.append((words, article))

## Build term-document matrix

In [318]:
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np

term_doc_matrix = np.zeros((len(words_list), len(vocab)), dtype=np.float32) # 메트릭스 만듬
for i, (words, article) in enumerate(new_words_list):
    for word in words:
        term_doc_matrix[i, word2id[word]] += 1
        
term_doc_matrix = TfidfTransformer().fit_transform(term_doc_matrix) #써이키런 -> 온갖 라이버러리 다 있다
print("Term document matrix shape:", term_doc_matrix.shape)
print(term_doc_matrix.toarray()[:10, :20])

Term document matrix shape: (671, 10000)
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.03813743
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.00991537
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0

## Topic modeling

In [287]:
# https://scikit-learn.org/stable/modules/classes.html#module-sklearn.decomposition
from sklearn.decomposition import NMF

K = 15  # topic 개수
nmf = NMF(n_components=K, alpha=0.1)  # Non-negative matrix factorization
W = nmf.fit_transform(term_doc_matrix) # 토픽모델링 해준다.
H = nmf.components_

print("W shape:", W.shape)
print("H shape:", H.shape)

W shape: (671, 15)
H shape: (15, 10000)


In [288]:
for k in range(K):
    print("{}th topic".format(k))
    for index in H[k].argsort()[::-1][:20]:
        print(vocab[index], end=' ')
        
    print("\n", "="*20)

0th topic
AI 데이터 개발 지능 신약 플랫폼 인공 기술 활용 서비스 솔루션 클라우드 정보 보안 기업 빅데이터 구축 분석 시스템 분야 
1th topic
NPU 전자 연산 반도체 핵심 기술 시스템 SoC 프로세서 모바일 AI 탑재 확대 삼성 LSI 사업 집중 계획 제품 병렬 
2th topic
TV LG 레드 전자 화질 영상 인치 해상도 QLED OLED 시장 출시 가전 판매 알파 화면 프리미엄 초대형 세대 프로세서 
3th topic
SK 텔레콤 에릭슨 스웨덴 GX 노키아 영상 사진 SKT 추억 슈퍼노바 협력 기술 마법 사진관 동영상 이벤트 개선 투표 선물 
4th topic
상담 챗봇 업무 고객 채팅 카드 국민카드 서비스 KB 처리 결제 금융 기능 질문 거래 큐디 대화 조회 디지털 저축은행 
5th topic
인텔 엔씨 엔씨소프트 디렉터 게임 블소 NC 양사 보영 퍼블리싱 데니스 루오 소프트웨어 비주 컴퓨팅 MOU 게이 시스 CPU PC 
6th topic
의료 수학 병원 수리 부산 협약 부산시 수리과학 심포지엄 유치 스케 기법 센터 연구 기업 해결 연구소 융합 부산대학교 순영 
7th topic
교육 청년 평택시 인재 양성 빅데이터 모집 미취 취업 채용 실습 신청 월간 사업 일자리 심사 산업혁명 맞춤 육성 참가자 
8th topic
학과 가천대 인공 지능 소프트웨어 대학 학년 교육 광운대 교수 신입생 번역 신설 양성 과정 개설 총장 인재 학교 커리큘럼 
9th topic
모비스 스타트업 선전 투자 기술 엠큐브 이노베이션 자율 현대 주행 혁신 미래 보유 중국 큐브 오픈 발굴 자동차 피터 현대차 
10th topic
공모전 아이디어 창업 환경부 환경 입찰 에코 미세먼지 부문 참가 정보 국민 상금 기획 정보통 수상자 입상 최종 재활용 활용 
11th topic
코딩 자격증 장학 교육 수강 진흥 이벤트 한국 인강 교육과정 빅데이터 지도사 응시 온라인 취득 무료 전문가 기초 시험 관심 
12th topic
네이버 랩스 AI 연구 기술 자율 주행 ICML 머

In [290]:
for k in range(K): # 토픽(15)에 대해 관련 기사를 나열
    print("{}th topic".format(k))
    for index in W[:, k].argsort()[::-1][:3]:
        print(words_list[index][1]['title'])
    print("\n", "="*20)

0th topic
대한민국 의약뉴스의 중심
빅데이터전문가(분석가), 코딩지도사 자격증 교육과정 무료수강 이벤트 운영
SK텔레콤, 스웨덴 현지에서 한-스웨덴 123년 교류 알렸다

1th topic
삼성전자, 독자 NPU로 ‘반도체 비전 2030’ 달성 가속화
‘기초 다지는 이재용’…‘시스템 반도체’ 이어 인공지능 본격화
삼성전자, AI시대 핵심 기술 ‘NPU’ 집중 육성한다-국민일보

2th topic
[라이프 트렌드] 바리스타·호텔리어·웨이터…우리는 로봇!
‘SKT 5GX 마법 사진관’ 투표 이벤트, 최고의 추억 사진은
도대체 얼마나 좋길래 TV가 5천만원?…LG전자, 8K 올레드 TV 국내 출시

3th topic
韓-스웨덴 123년 통신 협력 역사 넘어..SKT, 노키아·에릭슨과 5G 고도화 및 6G 진화 위해 손잡아
SK텔레콤, 음원서비스 ‘플로’ 멜론처럼 성장할까?
[스마트테크 코리아] 바벨탑 “클라우드 환경에서 번역 품질 실시간 확인”

4th topic
KB국민카드, 인공지능 챗봇 ‘큐티’ 론칭
KB국민카드, 인공지능 기반 챗봇 서비스 '큐디(Qd)' 선보여
KB국민카드, AI 기반 챗봇 서비스 '큐디' 선봬

5th topic
환경 공공정보 활용한 일자리 창출 공모전 열린다
삼성-LG, 거실에서 더 큰 화면으로 놀아봅시다
도쿄대, 간암 예측 인공지능 개발

6th topic
엔씨(NC), 인텔과 기술 협업 및 공동 마케팅 위해 맞손
미니TV 기능에 가족 얼굴도 인식… AI스피커, 보이는 ‘홈IoT센터’로 진화
수리과학연구소-부산광역시 업무협약

7th topic
평택시, 4차 산업혁명 핵심기술인 빅데이터 참여 청년교육생 모집
평택시 빅데이터 분석 전문가 교육생 모집
평택시, ‘빅데이터 분석 및 활용전문가 양성 과정’ 교육생 모집

8th topic
한국장학진흥원, 빅데이터전문가(분석가),코딩지도사 자격증 교육 무료수강 혜택 지원 이벤트
[네이버 테크인사이드] ③ 네이버는 왜 ‘자율주행’ 기술을 연구할까?
광주광역시, 한국의 AI 수도 노린다

9

## Visualization

In [296]:
from sklearn.manifold import TSNE

print("W shape:", W.shape)
tsne = TSNE(n_components=2, init='pca', verbose=1) 
W2d = tsne.fit_transform(W)
topic_index = [v.argmax() for v in W]  # 각각의 문서가 어느 topic에 속해 있는지 (hard clustering)

W shape: (671, 15)
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 671 samples in 0.002s...
[t-SNE] Computed neighbors for 671 samples in 0.029s...
[t-SNE] Computed conditional probabilities for sample 671 / 671
[t-SNE] Mean sigma: 0.031600
[t-SNE] KL divergence after 250 iterations with early exaggeration: 65.076088
[t-SNE] KL divergence after 1000 iterations: 0.564453


In [297]:
from bokeh.models import HoverTool
from bokeh.palettes import Category20
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource

output_notebook()

Loading BokehJS ...

In [303]:
tools_to_show = "hover,box_zoom,pan,save,reset,wheel_zoom"
p = figure(plot_width=720, plot_height=580, tools=tools_to_show)

source = ColumnDataSource(data={
    'x': W2d[:, 0],
    'y': W2d[:, 1],
    'id': [i for i in range(W.shape[0])],
    'document': [article['title'] for words, article in new_words_list],
    'topic': [str(i) for i in topic_index],
    'color': [Category20[K][i] for i in topic_index]
})

p.circle('x', 'y', source=source, legend='topic', color='color')

p.legend.location = 'top_left'
hover = p.select({'type': HoverTool})
hover.tooltips = [('Topic', '@topic'), ('id', '@id'), ('Article', '@document')]
hover.mode = 'mouse'

show(p)

## Siple search engine

In [316]:
query = 'SKT'
query_index = word2id[query]  # SKT 단어의 index

query_vec = term_doc_matrix[:, query_index]  # 각 문서에서 SKT 단어에 해당하는 값을 추출
nonzero_doc = query_vec.nonzero()[0]  # 0이 아닌 문서들만 사용
ranks = query_vec.toarray().flatten().argsort()[::-1]

for r in ranks[:10]:
    if r in nonzero_doc:
        print(new_words_list[r][1]['title'], new_words_list[r][1]['url'])


문대통령 북유럽 순방 발판…이통사-에릭슨·노키아 협력 '공고화' http://www.g-enews.com/ko-kr/news/article/news_all/201906160749523282bf45d5d5ea_1/article.html
‘SKT 5GX 마법 사진관’ 투표 이벤트, 최고의 추억 사진은 http://www.sporbiz.co.kr/news/articleView.html?idxno=343443
'사진으로 추억 소환하고 여행·치킨·커피까지' SKT, 마법사진관 이벤트 http://www.fntimes.com/html/view.php?ud=20190603104327544655469ec131_18
SKT "5GX 마법 사진관에서 추억을 뽑아보세요" http://www.asiatime.co.kr/news/articleView.html?idxno=249268
[뉴스워커_SK텔레콤] ‘SKT 5GX 마법 사진관’에서 내 마음을 사로잡은 추억을 뽑아보세요 http://www.newsworker.co.kr/news/articleView.html?idxno=30969
SK텔레콤, ‘SKT 5GX 마법사진관 No.1 픽’ 이벤트 시행 http://theleader.mt.co.kr/articleView.html?no=2019060410027828502
SK텔레콤, 'SKT 5GX 마법 사진관 No.1픽' 11일까지 진행 http://www.newsworks.co.kr/news/articleView.html?idxno=365108
"최고 추억사진 찾아라"…SKT, 5GX 마법 사진관 투표 http://www.inews24.com/view/1183413
​'대통령 북유럽 순방' SKT, 노키아·에릭슨과 5G이어 6G까지 ‘맞손’ http://www.ajunews.com/view/20190615084553021
과기정통부 등 관계부처 장관들, 5G 스마트오피스 현장방문 http://www.newsrep.co.kr/news/articleView.html?idxno=76876
